In [2]:
# 训练过程
#/home/wngys/lab/DeepFold/Code
import random
from torch.utils import data
from model import *
from data import *
from torch.utils.data import DataLoader
from torch import cosine_similarity
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn

In [4]:
# # 获取id对应的distance_matrix
def get_feature(dict_data, id, tfm):
    feature = torch.from_numpy(dict_data[id])
    feature = feature.to(torch.float)
    feature = tfm(feature)
    feature = feature.unsqueeze(0)
    return feature

In [5]:
class Max_margin_loss(nn.Module):
    def __init__(self, K, m) -> None:
        super().__init__()
        self.K = K
        self.m = m
   
    def forward(self, fingerpvec1, fingerpvec2, label):
        posi_vec_list = []
        nega_vec_list = []
        for number_inbatch in range(fingerpvec2.shape[0]):
            if label[number_inbatch] == 0:
                nega_vec_list.append(fingerpvec2[number_inbatch])
            elif label[number_inbatch] == 1:
                posi_vec_list.append(fingerpvec2[number_inbatch])
            else:
                print("ERROR")

        posi_cos_smi_list = []
        nega_cos_smi_list = []
        for posi_vec in posi_vec_list:
            # print("posi_vec: ", posi_vec.shape)
            posi_cos_smi_list.append(F.cosine_similarity(fingerpvec1, posi_vec, dim = 0))
        for nega_vec in nega_vec_list:
            nega_cos_smi_list.append(F.cosine_similarity(fingerpvec1, nega_vec, dim = 0))

        posi_cos_smi_list.sort() # 升序排序 选最小
        nega_cos_smi_list.sort(reverse=True) # 降序排序 选最大
        posi_cos = posi_cos_smi_list[0] # 只选取一个正例
        loss = 0
        for i in range(self.K):
            nega_cos = nega_cos_smi_list[i]
            loss += max(0, nega_cos - posi_cos + self.m)
        return loss

In [6]:
def valid_data(best_acc, valid_acc_ls, validIDlist, DFold_model, K=5):
        
    DFold_model.eval()
    acc_num = 0
    for id in validIDlist:
        feature1 = get_feature(dict_data, id, train_tfm)
        feature1 = feature1.to(device)

        id_list = get_id_list(valid_pair_dir + id +".txt")
        train_ds = Train_set(dict_data, id_list, train_tfm)
        train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)

        topKList = []
        for feature2, label in train_dl:
            with torch.no_grad():
                fingerpvec1 = DFold_model(feature1)
            feature2 = feature2.to(device)
            label = label.to(device)
            fingerBatch = fingerpvec1
            for b_i in range(feature2.shape[0] - 1):
                fingerBatch = torch.cat((fingerBatch,fingerpvec1), dim = 0)
            with torch.no_grad():
                fingerpvec2 = DFold_model(feature2)

            cos_smi_batch = F.cosine_similarity(fingerBatch, fingerpvec2, dim=-1)
            for cos_smi_idx in range(cos_smi_batch.shape[0]):
                if(len(topKList) < K):
                    topKList.append((cos_smi_batch[cos_smi_idx], label[cos_smi_idx]))
                else:
                    min_value = min(topKList)
                    if (cos_smi_batch[cos_smi_idx], label[cos_smi_idx]) > min_value:
                        min_idx = topKList.index(min_value)
                        topKList[min_idx] = (cos_smi_batch[cos_smi_idx], label[cos_smi_idx])
        acc_flag = False
        for _, label in topKList:
            if label == 1:
                acc_flag = True
        if acc_flag:
            acc_num += 1

    acc = acc_num / len(validIDlist)
    valid_acc_ls.append(acc)
    print(f"Epoch: {epoch} | acc_num: {acc_num} | total_num: {len(validIDlist)} | acc: {acc_num / len(validIDlist):.4f}")
    if acc > best_acc:
        best_acc = acc
        torch.save(DFold_model.state_dict(), "/home/wngys/lab/DeepFold/model/best_model.pt")
        print(f"saving best model with acc: {best_acc:.4f}")    
    DFold_model.train()
    return best_acc

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DFold_model = DeepFold(in_channel = 3)
DFold_model.to(device)

train_tfm = build_transform(in_channel = 3)
optimizer = torch.optim.SGD(DFold_model.parameters(), lr = 1e-3)
lossF = Max_margin_loss(K = 10, m = 0.1)

total_epochs = 10
batch_size = 64

train_loss_ls = []
valid_acc_ls = []

resume_dir = None
# resume_dir = "/home/wngys/lab/DeepFold/model/model_5.pt"
if resume_dir is not None:
    chkp = torch.load("/home/wngys/lab/DeepFold/model/model_5.pt")
    st_epoch = chkp["epoch"]
    best_acc = chkp["best_acc"]
    train_loss_ls.extend(chkp["train_loss_ls"])
    valid_acc_ls.extend(chkp["valid_acc_ls"])
    DFold_model.load_state_dict(chkp["model_param"])
    optimizer.load_state_dict(chkp["optim_param"])
else:
    st_epoch = 0
    best_acc = 0


pair_dir = "/home/wngys/lab/DeepFold/pair/train_pair_bool_90/"  
valid_pair_dir = "/home/wngys/lab/DeepFold/pair/pair_bool_90/"

trainIDlist = np.load("/home/wngys/lab/DeepFold/pair/train.npy", allow_pickle=True)
validIDlist = np.load("/home/wngys/lab/DeepFold/pair/valid.npy", allow_pickle=True)
random.shuffle(validIDlist)
validIDlist = validIDlist[:100]

dict_data = np.load("/home/wngys/lab/DeepFold/distance_matrix_r/matrix_data.npy", allow_pickle=True).tolist()


In [8]:
DFold_model.train()
total_train_loss = 0

In [9]:
random.shuffle(trainIDlist)

In [10]:
id_idx = None
id = None
for id_idx_, id_ in enumerate(trainIDlist[:800]):
    id_idx = id_idx_
    id = id_
    print(id_idx, id)
    break

0 d2gf5a2


In [12]:
print(id_idx, id)

0 d2gf5a2


In [13]:
feature1 = get_feature(dict_data, id, train_tfm)
feature1 = feature1.to(device)

In [15]:
print(feature1.shape)

torch.Size([1, 3, 256, 256])


In [ ]:
id_list = get_id_list(pair_dir + id +".txt")
train_ds = Train_set(dict_data, id_list, train_tfm)
train_dl = DataLoader(train_ds, batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [4]:
import torch

chkp = torch.load("/home/wngys/lab/DeepFold/new_model/new_model/model_2.pt")
print(chkp.keys())

dict_keys(['epoch', 'model_param', 'optim_param', 'train_acc', 'valid_acc', 'valid_id_list'])


In [5]:
print(chkp['train_acc'])

[(0.61, 61, 100), (0.68, 68, 100), (0.71, 71, 100), (0.75, 75, 100), (0.71, 71, 100), (0.77, 77, 100)]


In [6]:
print(chkp['valid_acc'])

[(0.63, 63, 100), (0.68, 68, 100), (0.76, 76, 100), (0.71, 71, 100), (0.75, 75, 100), (0.74, 74, 100)]
